# Exercises Week 12 - Association Rule Mining

In [1]:
import numpy as np

# Local imports
import sys
sys.path.append('./utilities')
from load_data import load_market_basket

## Exercise 1.

The work you do in this exercise will be very useful also in your hand-in.

We learned the Apriori algorithm in class. Make your own implementation. 

We will use the anonymized real-world retail market basket data from: http://fimi.ua.ac.be/data/.

This data comes from an anonymous Belgian retail store, and was donated by Tom Brijs from Limburgs Universitair Centrum, Belgium. The original data contains 16,470 different items and 88,162 transactions. You may only work with the top-50 items in terms of occurrence frequency.

Your task is to:
1. Implement the Apriori algorithm.
2. Apply the Apriori algorithm on these data to find association rules with minimum support 0.05 and minimum confidence 0.7. Write down the rules in descending order of confidence.


In [2]:
# Load the retail data
transactions = load_market_basket()

def book_example():
    return [
        [1, 2, 4, 5],
        [2, 3, 5],
        [1, 2, 4, 5],
        [1, 2, 3, 5],
        [1, 2, 3, 4, 5], 
        [2, 3, 4],
    ]

def filter_transactions(T, k=50):
    """
        Keep only the top k items in the transactions.
        Remove transactions that become empty.
    """
    # Count occurences of each item
    counts = [0] * 16470
    for t in T:
        for i in t:
            counts[i] += 1

    # Sort and select top k
    counts = np.array(counts)
    order  = np.argsort(counts)[::-1] # reverse the sorted order

    indexes_to_keep = order[:k]       # Keep the top k items
    index_set = set(indexes_to_keep)  # Convert to python set for efficiency

    # Filter transactions
    T_new = [t_ for t_ in  [list(filter(lambda i: i in index_set, t)) for t in T]  if t_]
    return T_new

T = filter_transactions(transactions, k=50)
#T = book_example() # You can first use the very small dataset to test

In [39]:
from itertools import combinations, chain

# Tiny function for generating rules from tuples
# Ex: rule((1, 2), (5)) outputs "(1, 2) => (5)"
rule  = lambda lhs, rhs: "%s => %s" % (str(lhs), str(rhs)) # For generating rule strings


def compute_support(Ck, T, still_applicable=None):
    counts = None
    ### TODO Your code here
    if still_applicable is None: still_applicable = [True] * len(T)
    
    # C1 = { x:0 for l in Ck for x in l}
    # for list1 in T:
    #     for i in list1:
    #         C1[i] += 1 #count
    

    counts = {}
    for row, t in enumerate(T):
        if not still_applicable[row]: continue #skip row if not applicable
        #flag for sayign if found anything still a subset
        found_any = False
        for c in Ck: #ck should be list of tuples
            if set.issubset(set(c),t):
                #increase if in dictionary else add
                if c in counts: counts[c] +=1
                else: counts[c] = 1
                found_any = True
        still_applicable[row] = found_any #if we found something then it is still usefull


    ### TODO Your code here
    return counts

def extend_prefix_tree(Ck_prev):
    Ck = None
    ### TODO Your code here
    Ck = set()
    #by making set we can ignore stuff with checking if we add twice ?

    #join step
    for itemset in Ck_prev:
        its1 = tuple(sorted(itemset)) #sort and make tuple
        for itemset2 in Ck_prev:
            #we want lex sorted such that we can use slicing
            its2 = tuple(sorted(itemset2))
            #check if same uptii k-1 and that last is not same
            if its1[:-1] == its2[-1] and its1[-1] < its2[-1]: # its1>its2 would in the end do the same
                # its1 and its2 are a tuple, 
                Ck.add(its1 + its2[-1:])
                #could have cast them to sets, used union, and then casted them back to tuplles
    #now prune
    to_remove = set()
    for c in Ck:
        for subset in combinations(c, len(c)-1): #every comb execpt c?
            if not subset in Ck_prev:
                to_remove.add(c)
                break #we dont need to check other subsets

    for c in to_remove:
        Ck.remove(c)

    ### TODO Your code here
    return Ck #now we have all sets where all subsets are present?

def powerset(iterable):
    ### TODO Your code here
    s = list(iterable)
    # we want any sets at most the lengt of the list
    return chain.from_iterable(combinations(s,r) for r in range(1, len(s)))
    ### TODO Your code here

def apriori_algorithm(T, support=0.05, min_confidence=0.7):
    """
        Apriori algorithm for mining association rules.
        Inputs:
            T:               A list of lists, each inner list will contiain integer-item-ids. 
                             Example: T = [[1, 2, 5], [2, 3, 4], [1, 6]]
            support:         The proportion of occurences needed to keep itemsets.
            min_confidence:  Minimum confidence for the algorithm to output the rule.
        
        Outputs:
            rules:           List of tuples [(rule:str, confidence:float), ... ]
                             Example: [("(1, 2) => (5)", 0.84), ("(3, 4) => (7)", 0.75)]
    """
    rules = []
    ### TODO Your code here
    n = len(T)
    
    itemsets = {}
    C1 = set()
    for t in T:
        for ti in t: C1.add((ti,)) #add all items as a tuple

    still_applicable = [True] * n
    Ck = C1

    k=1
    while Ck:
        itemsets[k] = compute_support(Ck, T, still_applicable)
        Ck_copy = Ck.copy()
        for itemset in Ck:
            #
            if itemsets[k][itemset] / n < support:
                del itemsets[k][itemset]
                Ck_copy.remove(itemset)
        Ck = extend_prefix_tree(Ck_copy) #want to look at the next itemsets of k+1
        #terminates at some point because we prune and stuff
        k+=1
    
    k=2 #we dont care about rules of size 1
    rules= []

    while k <= max(itemsets.keys()):
        for itemset in itemsets[k].keys():
            for rhs in powerset(itemset):
                remaining = set(itemset).difference(set(rhs))
                lhs = tuple(sorted(remaining))

                conf = itemsets[k][itemset] / itemsets[len(lhs)][lhs]
                if conf >= min_confidence:
                    rules.append((rule(lhs,rhs),conf))

        k +=1
        # Use Lk to generate candidates of size k+1

    ### TODO Your code here
    return rules

Below you can try your algorithm. If you want, you could try changing the support and min_confidence to see what happens to the result.

In [40]:
rules = apriori_algorithm(T, support=0.05, min_confidence=0.7)
rules = sorted(rules, key=lambda x: x[1], reverse=True)

print("%-8s \t %s" % ("Conf.", "Rule"))
for r in rules:
    print("%7.4f%% \t %s" % r[::-1])

Conf.    	 Rule


## Exercise 2.

We have learned how to mine frequent itemsets and association rules from a transaction database where each transaction consists of a simple set of items. You are asked to propose a framework for mining association rules from transaction data, in which each item in a transaction is associated with an integer number that counts how many times the items appears in the transaction. In a market basket, this count number indicates the number of copies of a product in a customer’s basket. For example, we do not only care whether a customer bought fish or not, but how many pieces of fish they bought.

1. Define (extend) the notion of an itemset and an association rule in the case of such data.
2. Describe an efficient algorithm that mines itemsets and association rules as defined in Exercise 2. Illustrate the pruning strategies used in your algorithm and explain how they relate to the Apriori principle.
3. Extend your implementation of the Apriori algorithm to handle this case.

In [ ]:
### TODO your code here
# You can copy pase the code from above and adjust it
### TODO your code here

Keep dictionary for the items such that we also have count  
Need to not just looup if item is present or not, but also how many then  
  
Differentiate between (A,1) => (B,1) and (A,2) => (B,3)  
We still want to keep some conection between A's and B's 
  
((A,2),(C,3) => (D,1)) different from (A,1),(C,3) is very strict, maybe limiting  

Could also change notion of support. E.g if we have rule AC => D and have (A,3) (C,5) (D,4) would then support that rule 3 times.  

Could also compute mean of all support of transactions  
The mean of A2C3 => D1 and A1C3=>D2 would be A1.5 C3 => D1.5
Can use mean to estimate how many items in the end?... 
Would end with 
Doesn't need to change the underlying algorithm, just need to also keep track of mean.
Would loose on the threshold? Buying for large party would skew?  
For an idividual perspective the mean would be useless.  



What are our use-cases. <- the thing it all comes down to

## Exercise 3

Consider a dataset of transactions $D$, and let $D'$ be a dataset derived from $D$ by independently deleting items from transactions in $D$. In particular, any item in any transaction in $D$ is deleted with probability $p$.

1. Given an itemset $S$, compute its expected support in $D'$ as a function of its support in $D$.
2. Compute the probability that an itemset $S$, which is frequent in $D$, is also frequent in $D'$, under the same minimum support threshold.


$\sigma(I) = |{t\in T | I \subseteq t(D)}|$


Hvis vi har {{A,B},{A,B},{A}} og det sidste A slettes så stiger support for reglen A=>B  

Hvis vi har {{A,B},{A,B},{A}} og det sidste A slettes falder support for A og support for B er uændret

$|s|=k$  

$T_i$

$s\leq i(t_i)$

We need to figure out the prop of not being deleted.
$Pr[not deleting any x \in s from T_i]$

$\Pi_{x\in s} (1-p) = (1-p)^k$

$E[] = \sum_{t_i \in D} (1-p)^k*1 = (1-p)^k * S_{n,p?}(S,D)$

Second is bionominal distribution, but otherwise much of the same principle